In [ ]:
# Automatic Dataset Download for Color Normalization Tutorial
import os
import requests
from pathlib import Path
from tqdm import tqdm
import json

def download_color_normalization_data():
    """Download diverse histopathology samples with different staining variations"""
    data_dir = Path("../data")
    color_samples_dir = data_dir / "color_samples"
    color_samples_dir.mkdir(parents=True, exist_ok=True)
    
    # Diverse staining samples for color normalization
    staining_samples = {
        'he_stain_1.jpg': 'https://github.com/jamesdolezal/slideflow/raw/master/docs/source/_static/img/tile_5.jpg',
        'he_stain_2.jpg': 'https://github.com/jamesdolezal/slideflow/raw/master/docs/source/_static/img/tile_6.jpg',
        'reference_normal.png': 'https://raw.githubusercontent.com/PathologyDataScience/TCGA-BRCA-WSI-Tiles/main/sample_tiles/normal_002.png',
        'target_tissue.png': 'https://raw.githubusercontent.com/PathologyDataScience/TCGA-BRCA-WSI-Tiles/main/sample_tiles/tumor_001.png'
    }
    
    # Color normalization parameters for different methods
    normalization_params = {
        'macenko': {
            'luminosity_threshold': 0.8,
            'alpha': 1.0,
            'beta': 0.15
        },
        'reinhard': {
            'target_mu': [8.74108109, -0.12440419,  0.0444982],
            'target_sigma': [0.6135447, 0.10989545, 0.0286032]
        },
        'vahadane': {
            'lambda1': 0.01,
            'lambda2': 0.01,
            'fast_mode': True
        }
    }
    
    # Save normalization parameters
    params_file = color_samples_dir / "normalization_params.json"
    with open(params_file, 'w') as f:
        json.dump(normalization_params, f, indent=2)
    print(f"📊 Saved normalization parameters to {params_file}")
    
    print("🎨 Downloading color normalization samples...")
    download_success = []
    
    for filename, url in staining_samples.items():
        filepath = color_samples_dir / filename
        if not filepath.exists():
            try:
                print(f"📥 Fetching {filename}...")
                response = requests.get(url, stream=True, timeout=30)
                response.raise_for_status()
                
                total_size = int(response.headers.get('content-length', 0))
                with open(filepath, 'wb') as f:
                    if total_size > 0:
                        with tqdm(total=total_size, unit='iB', unit_scale=True, desc=filename) as pbar:
                            for chunk in response.iter_content(chunk_size=8192):
                                if chunk:
                                    f.write(chunk)
                                    pbar.update(len(chunk))
                    else:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                
                print(f"✅ Downloaded {filename} successfully")
                download_success.append(filename)
            except Exception as e:
                print(f"❌ Error downloading {filename}: {e}")
                # Create synthetic sample for demo purposes
                try:
                    import numpy as np
                    from PIL import Image
                    synthetic = np.random.randint(100, 200, (512, 512, 3), dtype=np.uint8)
                    # Add some structure to make it look more tissue-like
                    synthetic[100:150, 100:150] = [180, 120, 160]  # Purple-ish region
                    synthetic[300:400, 200:350] = [220, 180, 190]  # Pink-ish region
                    Image.fromarray(synthetic).save(filepath)
                    print(f"🎭 Created synthetic sample for {filename}")
                except Exception:
                    pass
        else:
            print(f"✅ {filename} already exists")
            download_success.append(filename)
    
    print(f"🎯 Color normalization data ready! {len(download_success)}/{len(staining_samples)} samples available")
    print(f"📁 Data location: {color_samples_dir.absolute()}")
    return color_samples_dir, normalization_params

# Initialize color normalization data
color_data_dir, norm_params = download_color_normalization_data()
print("🔬 Ready for color normalization experiments!")

## Color Normalization

Histopathology images can have significant color variations due to differences in staining and scanning. Color normalization aims to reduce this variation.

**Our Goals:**
1.  Understand the need for color normalization.
2.  Implement a simple stain normalization technique.

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# For this example, we'll create two "stained" images to simulate variation.
# In a real scenario, you would use real image tiles.
def create_stained_image(r_stain, g_stain, b_stain):
    # Create a base image (e.g., representing tissue)
    base = np.ones((100, 100, 3), dtype=np.uint8) * 250
    base[20:80, 20:80, :] = [200, 150, 200] # A "tissue" area
    
    # Apply "stain"
    stained = base.astype(np.float32)
    stained[:,:,0] *= r_stain
    stained[:,:,1] *= g_stain
    stained[:,:,2] *= b_stain
    
    return Image.fromarray(np.clip(stained, 0, 255).astype(np.uint8))

source_image = create_stained_image(0.9, 0.7, 0.85)
target_image = create_stained_image(1.0, 0.8, 0.7)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(source_image)
axes[0].set_title('Source Image')
axes[1].imshow(target_image)
axes[1].set_title('Target Image (Reference)')
plt.show()

### 1. Simple Normalization using Mean and Standard Deviation

A common method is to scale the source image's color channels to match the mean and standard deviation of a target (reference) image.

In [ ]:
def normalize_color(source, target):
    source_arr = np.array(source, dtype=np.float32)
    target_arr = np.array(target, dtype=np.float32)
    
    normalized_arr = np.zeros_like(source_arr)
    
    for i in range(3): # For each channel (R, G, B)
        source_channel = source_arr[:,:,i]
        target_channel = target_arr[:,:,i]
        
        # Get stats
        src_mean, src_std = np.mean(source_channel), np.std(source_channel)
        tgt_mean, tgt_std = np.mean(target_channel), np.std(target_channel)
        
        # Normalize
        normalized_channel = (source_channel - src_mean) / src_std * tgt_std + tgt_mean
        normalized_arr[:,:,i] = normalized_channel
        
    return Image.fromarray(np.clip(normalized_arr, 0, 255).astype(np.uint8))

normalized_image = normalize_color(source_image, target_image)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(source_image)
axes[0].set_title('Source')
axes[1].imshow(target_image)
axes[1].set_title('Target')
axes[2].imshow(normalized_image)
axes[2].set_title('Normalized')
plt.show()

## ✅ Final Check

Let's check the mean of the normalized image's channels. They should be closer to the target's channel means.

In [ ]:
source_means = np.mean(np.array(source_image), axis=(0,1))
target_means = np.mean(np.array(target_image), axis=(0,1))
normalized_means = np.mean(np.array(normalized_image), axis=(0,1))

print(f"Source means: {source_means}")
print(f"Target means: {target_means}")
print(f"Normalized means: {normalized_means}")

# Check if normalized means are closer to target means
assert np.all(np.abs(normalized_means - target_means) < np.abs(source_means - target_means))

print("\nSUCCESS: Normalized image stats are closer to the target.")